In [1]:
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I cannot provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 17, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a1ea1184-85dd-48f9-ac4f-dc8398f0059e-0')

In [6]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RXpYLFaL9WXLeImPnvBRGSho', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 194, 'total_tokens': 211}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cbcd2a79-24ad-4c65-a99f-803fc9ef334e-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_RXpYLFaL9WXLeImPnvBRGSho'}])

In [9]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? I would like to eat outside if possible"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zSnzs2dgx4Jr4PXabOXDnxax', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_7zdqwpJcYHlEmuCRPtpIuyYv', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 202, 'total_tokens': 254}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6ca46cd-4018-4d17-8f50-2aa35851543c-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_zSnzs2dgx4Jr4PXabOXDnxax'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_7zdqwpJcYHlEmuCRPtpIuyYv'}])

In [10]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_zSnzs2dgx4Jr4PXabOXDnxax'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_7zdqwpJcYHlEmuCRPtpIuyYv'}]

In [11]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [12]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BrvOv4sHQ2TLKwuAkcCJUrUK', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_IrLtfuYkf5uEzNrGgQdRfrOS', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 202, 'total_tokens': 254}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-836794cc-094d-41d5-b7bb-204a8b081e7f-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_BrvOv4sHQ2TLKwuAkcCJUrUK'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_IrLtfuYkf5uEzNrGgQdRfrOS'}])]

In [13]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [14]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BrvOv4sHQ2TLKwuAkcCJUrUK', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_IrLtfuYkf5uEzNrGgQdRfrOS', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 202, 'total_tokens': 254}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-836794cc-094d-41d5-b7bb-204a8b081e7f-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_BrvOv4sHQ2TLKwuAkcCJUrUK'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_IrLtfuYkf5uEzNrGgQdRfrOS'}]),
 ToolMessage(content='Sunny, 22°C', tool_call_id='cal

In [16]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Outdoor seating is available, so you can enjoy your meal outside.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 282, 'total_tokens': 311}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d34f1296-10f0-432f-931f-1029528c2118-0')

### Real third party API Call

In [17]:
from langchain_core.tools import tool
import httpx


@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/weather?city={city}")

    if response.status_code == 200:
        return response.json().get("weather", "Weather information not available")
    else:
        return "Failed to get weather information"


@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/outdoor-seating?city={city}")

    if response.status_code == 200:
        return response.json().get(
            "outdoor_seating", "Outdoor seating information not available"
        )
    else:
        return "Failed to get outdoor seating information"


api_tools = [fake_weather_api, outdoor_seating_availability]
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [18]:
from langchain_openai import ChatOpenAI


def interact_with_llm_and_tools(human_message: str):

    llm = ChatOpenAI()
    llm_with_tools_new = llm.bind_tools(api_tools)

    messages = [human_message]

    llm_output = llm_with_tools_new.invoke(messages)
    messages.append(llm_output)

    for tool_call in llm_output.tool_calls:
        tool_name = tool_call["name"].lower()
        tool = tool_mapping.get(tool_name)

        if tool:
            tool_output = tool.invoke(tool_call["args"])
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm_with_tools_new.invoke(messages)
    return final_response

In [20]:
interact_with_llm_and_tools(
    "How will the weather be in sunnyville today? I would like to eat outside if possible"
)

AIMessage(content="I'm sorry, but I couldn't retrieve the weather information for Sunnytown or check the availability of outdoor seating. Is there anything else I can assist you with?", response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 213, 'total_tokens': 247}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b9ae449-2d34-4d44-8314-c478cadbd2ef-0')

In [ ]:
interact_with_llm_and_tools(
    "How will the weather be in rainytown today? I would like to eat outside if possible"
)

In [ ]:
interact_with_llm_and_tools(
    "How will the weather be in munich today? I would like to eat outside if possible"
)

In [ ]:
interact_with_llm_and_tools("What´s the name of the Dad of the simpsons family?")